# Introduction to MPI - Day 1

In [ ]:
import os

# the jupyter notebook is launched from your $HOME, change the working directory provided a username directory is created under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/mpi/day1"))

## 0. Preliminary Example
Our first example ([MC_pi.c](./MC_pi.c)) for you to get a hang of MPI code is slightly more than a helloword program. Nevertheless, it is a simple snippet showcasing a basic MPI program template. 

The program approximates Pi by Monte-Carlo method. In essence, $N$ number of randowm numbers are distributed to multiple processes. Each process independtely calculates the number of random numbers that are inside a unit circle and the results are collected and summed at a root process (rank 0). 

Look out for the following parts in the program.
```cpp
#include <mpi.h>

MPI_Init(&argc, &argv);

MPI_Comm_rank(MPI_COMM_WORLD, &rank);

MPI_Comm_size(MPI_COMM_WORLD, &size);

MPI_Wtime();

MPI_Reduce(&count, &count_tot, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

MPI_Barrier(MPI_COMM_WORLD);

MPI_Finalize();
```

Run the next cell to excute the MC_pi program.

In [ ]:
!make clean && make MC && echo "Compilation Successful!" && mpiexec -np 8 ./MC_pi

## 1. Code Description

### 1.1 The model problem
The code is a numerical solver for 2D Poisson equation with Dirichlet boundary condition discretised by 2nd order central finite difference operator on a unit square domain. 

Consider 

\begin{align*}
-\Delta u &= f \; \text{in} \; \Omega \\
 u &= g \; \text{on} \; \partial \Omega
\end{align*}

for $\Omega = [0,1] \times [0,1]$.

Define a uniform partition of the domain $\Omega$ with nodal points at which the solution of the Poisson equation is sampled. Let $h$ be the uniform distance between two nodal points then the nodal points that lie on the mesh are defined by

\begin{align*}
x_i = i h, \; y_j = j h\qquad i,j = 0,\cdots, N
\end{align*}
where $N$ is a given mesh size and $i, j$ are integers along $x, y$-axis telling the location of each nodal point.  

### 1.2 Discretisation
We use the second-order central finite difference method to discretise the Laplace operator
 \begin{align*}
     \big(\Delta u)\big)_{i,j} & = \big(D_{xx}^2u\big)_{i,j}+ \big(D_{yy}^2 u\big)_{i,j}\\[2ex]
     & \approx \frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{h^2} + \frac{u_{i,j+1}-2u_{i,j}+u_{i,j-1}}{h^2} 
 \end{align*}

leading to
\begin{align}
    -\big(\Delta u)\big)_{i,j} = \frac{4u_{i,j}-u_{i+1,j}-u_{i-1,j}-u_{i,j+1}-u_{i,j-1}}{h^2}=f(u_{i,j}).
\end{align}

The above finite-difference formula can further be represented by a five-point stencil matrix built in the mesh
\begin{align*}S = 
    \begin{pmatrix}
    & -1 & \\
    -1 & 4 &-1\\
    & -1 &
    \end{pmatrix}.
\end{align*}

Impose the Dirichlet boundary condition, on the interior nodal points the discretisation can be written as a linear equation

\begin{align*}
Au = f, \qquad A=\frac{1}{h^2}
    \begin{pmatrix}
S & I \\
I & S & I \\
& I & \ddots & \ddots \\
& & \ddots & \ddots & I \\
& & & I & S
\end{pmatrix}
\end{align*}
where $A \in \mathbb{R}^{(N-2)^2 \times (N-2)^2}$, $u \in \mathbb{R}^{(N-2)^2}$ and $f \in \mathbb{R}^{(N-2)^2}$.

Note that with the five-point stencil, the matrix $A$ was never assembled and is nowhere in sight! 

### 1.3 Numerical Solvers

To solve the linear system, the Jacobi iterative method is used and compared. 

$\textbf{Jacobi method}$

\begin{align*}
u^{(k+1)} = D^{-1}( f - Lu_{k} -Uu_k),
\end{align*}
where $D, L, U$ are the diagonal matrix, lower triangular matrix and upper triangular matrix of $A$, respectively.
Write into stecil,
\begin{align*}
u^{(k+1)}_{ij} = (h^2 f_{ij} + u^{(k)}_{i-1,j} +u^{(k)}_{i+1,j} + u^{(k)}_{i, j-1}+u^{(k)}_{i,j+1})/4 \qquad i,j = 1,\cdots, N-1 
\end{align*} 

The method is implemented in file [solver.c](./solver.c) if you want a peak but remember no refactoring is needed for routines in this file. 

### 1.4 Convergence measurement 
Usually, it is impossible to measure the error of the numerical solver as it requires to know the true solution $e^{(k)}= u^{(k)} -u^\ast$ where $u^\ast$ is unkown. 

Often the measurement of the differene between appproximation is seen, i.e. $\lim_{k\to \infty} d^{(k)} = u^{(k)} - u^{(k-1)} = 0$. However this cauchy sequency type of quantity has no implication of the convergence to the matrix problem. Therefore this practice should be avoided.

A standard measurement is by the residual of the problem,
$$r^{(k)} = f - Au^{(k)}$$
as implemented in [solver.c](./solver.c), owing to the following relation
$$Ae^{(k)} = r^{(k)} \implies  \|r\|\le \|A\| \|e\|.$$
Implies $$\|e^{(k)}\| \le \|A^{-1}\| \|r^{(k)}\|,$$
that is $$\lim_{k\to \infty} \|r^{(k)}\| = 0 \implies \lim_{k\to \infty}\|e^{(k)}\| = 0.$$

Note the Jacobi method itself is not practical to use for large-size problems due to a poor convergence rate. In fact, one can show the convergence rate is
$$\mathcal{O}(1-h^2).$$
As such, you are strongly advised to use more efficent methods for your problems derived from real-life applications, such as, preconditioned conjugate gradient, Krylov methods, multigrid method etc. 

Notwithstanding the unfavourable numerical aspect, the method is adapted in our workshop to demonstrate the parallelisation practice.



### 1.5 Parallelisation
As disscused in the slides, the unit square domain is decomposed into to subdomains for parallelisation. Each MPI process hosts a subdomain. 
```cpp
double (*submesh)[mesh_size] = malloc(sizeof *submesh * *ptr_rows);
```
where mesh_size is the number of grid points per dimension. 


The decomposition is performed vertically, resulting two rows of ghost nodes on the top and the bottom for each subdomain.
```cpp
/* top row of ghost nodes */
submesh[*ptr_rows -1]

/*bottom row of ghost nodes */
submesh[0]
```
The mesh initialisation is written in [mesh.c](./mesh.c). Please do not edit.

Note that the value that the pointer ptr_rows pointing to is different for the top slab subdomain as it may host extra number of rows. The following example demonstrates the decomposition for a domain of 27x27 distributed to 4 MPI processes. 

## 2. Blocking Communication

Each subdomain needs to update its ghost rows from neighbouring processes and send the top and bottom rows of full nodes to the neighbouring processes. 

For a MPI process $i$ to recv the update on the top row of ghost nodes with a standard blocking communication, we use
```cpp 
/* on MPI process i */
 MPI_Recv(submesh[*ptr_rows -1], mesh_size, MPI_DOUBLE, upper, highertag, MPI_COMM_WORLD, &status);
```

This requires the upper rank $i+1$ to coordinate with a send call
```cpp
/* on MPI process i+1 */
MPI_Send(submesh[1], mesh_size, MPI_DOUBLE, lower, highertag, MPI_COMM_WORLD);
```

Because every processes will execute those two calls, they can be put together without specifying the rank. See [laplace_mpi_blocking.c](./laplace_mpi_blocking.c).

**`TODO`**: in [laplace_mpi_blocking.c](./laplace_mpi_blocking.c), only top ghost row is updated, complete the same communication for the bottom ghost row. Once you are finished run the next cell. If you are stuck, peek solution at [soln_laplace_mpi_blocking.c](./solution/laplace_mpi_blocking.c)

Essential MPI functions needed: `MPI_Recv`, `MPI_Send`.

In [ ]:
!make clean && make blocking && echo "Compilation Successful!" && mpiexec -np 12 ./laplace_mpi_blocking 300 1000 Jacobi > convergence_blocking.txt
!tail -20 convergence_blocking.txt 

## Nonblocking Communication
During the communication between MPI processes, the iterative method can perform on interior grid nodes (except top and bottom full nodes) concurrently. To this end, a `Jacobi_int` routine is used and separated from `Jacobi_top` and `Jacobi_bottom` (see [solver.c](./solver.c)).

Hence, the communication starts from sending and receiving data for the top and bottom rows, and is followed by the `Jacobi_int` routine. Before applying `Jacobi_top` and `Jacobi_bottom` respectively, one needs to ensure the corresponding communication call is completed. The communications use `MPI_Request`
```cpp
 MPI_Request top_bnd_requests[2],  bottom_bnd_requests[2];
```
to identify the communications. 

In the code [lapalce_mpi_nonblocking.c](./laplace_mpi_nonblocking.c), it is written based on the "first in first served" concept - whichever request is completed, it gets to proceed with the Jacobi method. 
```cpp
    /* Test on either the top or bottom layer */
    if ( (MPI_Testall(2, top_bnd_requests, &top_flag, top_bnd_status) > 0) || (MPI_Testall(2, bottom_bnd_requests, &bottom_flag, bottom_bnd_status) > 0))
    {
       MPI_Abort(MPI_COMM_WORLD, 1);
    }

    /* if the top layer is ready */
    if (top_flag){
        /* perform jacobi on the top bnd */
        Jacobi_top(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);

    /* if the the bottom layer is ready */
    if (bottom_flag){
    /* perform jacobi on the bottom bnd */
    Jacobi_bottom(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);
    }
    
    /* if the bottom layer is yet ready */
    else{
        /* wait on the bottom layer */
        MPI_Waitall(2, bottom_bnd_requests, bottom_bnd_status);
        Jacobi_bottom(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);        
        }
    }
    /* if the top layer is yet ready but the buttom is */
    else if (bottom_flag){
        /* perform jacobi bottom ready */
        Jacobi_bottom(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);
        /* wait on the top layer */
        MPI_Waitall(2, top_bnd_requests, top_bnd_status);
        Jacobi_top(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);
        }
    /* if neither of the top and bottom is ready, then wait on both */
    else {
        MPI_Waitall(2, bottom_bnd_requests, bottom_bnd_status);
        Jacobi_bottom(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);
        MPI_Waitall(2, top_bnd_requests, top_bnd_status);
        Jacobi_top(ptr_rows, mesh_size, &submesh[0][0], &submesh_new[0][0], &subrhs[0][0], space);
    }
```
However, this may overcomplicated the program. 

**`TODO`**  Change the code in [laplace_mpi_nonblocking.c](./laplace_mpi_nonblocking.c) to a simplifed version that binds both top and bottom communications to a single array of requests. Once you are finished run the next cell. If you are stuck, peek solution at [soln_laplace_mpi_nonblocking.c](./solution/laplace_mpi_nonblocking.c)

Essential MPI functions needed: `MPI_IRecv`, `MPI_ISend`, `MPI_Testall`, `MPI_Waitall`.



In [ ]:
!make clean && make nonblocking && echo "Compilation Successful!" && mpiexec -np 12 ./laplace_mpi_nonblocking 300 1000 Jacobi > convergence_nonblocking.txt
!tail -20 convergence_nonblocking.txt 

## 3. Persistent Communication

You should now be quite familiar with our code, and may notice that the same communication is executed over and over again only with different data buffer within the iteration loops. Recall the four stages of `MPI Operation` mentioned in the slides. The `initialisation stage` hands over the argument list, which stays unchanged in our program. The persistent communication binds the argument list to the communication request once and repeatedly uses it in the subsequent communication calls.  

**`TODO`** Refactor the code in [laplace_mpi_persistent.c](./laplace_mpi_persistent.c), which is using previous nonblocking routines at the current status, to a persistent communication. Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_persistent.c](./solution/laplace_mpi_persistent.c).

Essential MPI functions needed: `MPI_Recv_init`, `MPI_Send_int`, `MPI_Startall`, `MPI_Waitall`, `MPI_Request_free`

In [ ]:
!make clean && make persistent && echo "Compilation Successful!" && mpiexec -np 12 ./laplace_mpi_persistent 300 1000 Jacobi > convergence_persistent.txt
!tail -20 convergence_persistent.txt